In [1]:
!pip install pandas matplotlib requests networkx 

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/fb/4f/4a4372b2e24439f559b73318683486831d75e59544ae02bf8dec8dd6f48b/pandas-2.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/c2/da/a5622266952ab05dc3995d77689cba600e49ea9d6c51d469c077695cb719/matplotlib-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.6 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for numpy>=1.22.4 from https://files.pythonhosted.org/packages/71/3c/3b1981c6a1986adc9ee7db760c0c34ea5b14ac3da9ecfcf1ea2a4ec6c398/numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 7.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 7.8 MB/s eta 0:00:00a 0:00:01
 

This notebook is used for preparing shipping paths for the Welland Lock animation.

Required files
1. '1854 WCR 2023.csv'-> Historical Shipping CSV derived from spreadsheet file
2. 'water_paths.json' -> Shipping paths converted to geojson with QGIS. Derived from this: https://www.arcgis.com/home/item.html?id=a4940deebec84fb9b6afa65afcbf891d#overview

Output
1. manifest.json -> Paths that the animation will use to illustrate ship voyages

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import json
import requests
import networkx as nx
import urllib.parse
from datetime import date
from math import radians, sin, cos, atan2, sqrt, isnan
from pprint import pprint

Read in CSV data, clean it up, and prepare the first section of the json data that will be exported.
This first section is the csv converted to json format. 

In [4]:
df = pd.read_csv('1854 WCR 2023.csv',header=1).drop(columns=['Column1','Direction'])
df['Where Bound'] = df['Where Bound'].apply(lambda x: str(x).strip())
df['Where From'] = df['Where From'].apply(lambda x: str(x).strip())

#exclude rows where locations have not be filled in or lon/lat could not be determined
exclude_list = ['?','','nan','Bear Creek','bear Creek','Miloin','Port Price']

df = df[(~df['Where Bound'].isin(exclude_list)) & (~df['Where From'].isin(exclude_list))]

def not_empty(item):
    if type(item) == float and isnan(item):
        return False
    elif type(item) == type(None):
        return False 
    elif type(item) == str and item.strip() == '':
        return False
    else: 
        return True

cargo = []
for i,v in df[['Cargo 1', 'Cargo 2', 'Cargo 3', 'Cargo 4']].iterrows():
    cargo_this = map(str.strip, filter(not_empty, [v['Cargo 1'], v['Cargo 2'], v['Cargo 3'],v['Cargo 4']]))
    cargo.append(list(cargo_this))
df['Cargo'] = cargo
df = df.drop(columns=['Cargo 1', 'Cargo 2', 'Cargo 3', 'Cargo 4'])

#Check values are clean
assert (df['Year'].apply(lambda x: x == 1854.0).all())
assert df['Day'].apply(lambda x: x>=1 and x<=31).all()

for col in ['Nationality', 'Vessel Type', 'Name of Vessel']:
    df[col] = df[col].apply(str.strip)

month_to_int = {"January":1, "Febuary":2, "March": 3, "April": 4, "May": 5, 
 "June":6, "July":7, "August":8, "September":9,"October":10,"November":11, "December":12}
df['Date'] = df[['Year','Month','Day']].apply(lambda x: str(date(int(x.Year), month_to_int[x.Month], int(x.Day))), axis=1)    
df.drop(columns=['Year','Month','Day'], inplace=True)

Show some information about the dataset

In [7]:
print(set(df['Nationality']))
print(set(df['Vessel Type']))

#Total number of days
print(len(df['Date'].drop_duplicates()))

{'American', 'British'}
{'Barkentine', 'Schooner', 'Steamer', 'Propeller', 'Brigantine', 'Brigantineantine', 'Sail Boat'}
149


Read in the shipping path data that was converted to json using QGIS

In [8]:
with open('water_paths.json') as wp_geojson:
    gj = json.loads(wp_geojson.read())
    

Define functions for transforming the path information into a graph and working with that graph

In [11]:
def feature_filter(gj, gid_list=None, exclude=True):
    '''filter out features from geojson. optionally exclude or include only features with gid in gid_list.'''
    filtered = []
    for f in gj['features']:
        if not gid_list:
            filtered.append(f)
            continue
            
        if exclude and f['properties']['gid'] not in gid_list: 
            filtered.append(f)
        elif not exclude and f['properties']['gid'] in gid_list: 
            filtered.append(f)
    
    return filtered

def distance(p1, p2):
    '''returns distance in KM between two points'''
    earth_radius = 6371
    lat1, lat2 = radians(p1[1]), radians(p2[1])
    lon1, lon2 = radians(p1[0]), radians(p2[0])
    
    #https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    base = earth_radius * c
    return base

def draw_graph(G):
    pos = nx.planar_layout(G)
    nx.draw(G, pos, with_labels=True, font_weight='bold')
    el=nx.get_edge_attributes(G,'weight')
    nx.draw_networkx_edge_labels(G,pos,edge_labels=el)

def path_distance(g, path):
    prev = path[0]
    distances = []
    for p in path[1:-1]:
        distances.append(g.edges[(prev,p)]['weight'])
        prev = p
    p = path[-1]
    distances.append(g.edges[(prev,p)]['weight'])
    return sum(distances)

def geojson_to_graph(features):
    G = nx.Graph()
    for feat in features:
        assert len(feat['geometry']['coordinates']) == 1
        feat_geo = feat['geometry']['coordinates'][0]
        anode = str(feat['properties']['anode'])
        bnode = str(feat['properties']['bnode'])
        #length = feat['properties']['length']
        id = feat['properties']['OBJECTID']
        id_postfix = 1
        G.add_node(anode, coordinates=tuple(feat_geo[0]))
        G.add_node(bnode, coordinates=tuple(feat_geo[-1]))

        prev = anode
        prev_coord = feat_geo[0]
        for point in feat_geo[1:-1]:
            name = f"{id}_{id_postfix}"
            id_postfix += 1
            G.add_node(name, coordinates=tuple(point))
            G.add_edge(prev,name, weight=distance(prev_coord, point))
            prev = name
            prev_coord = point
        G.add_edge(prev, bnode, weight=distance(prev_coord, feat_geo[-1]))
    return G

def coordinates_set(g):
    return {g.nodes[node]['coordinates'] for node in g.nodes}

def closest_coordinate(all_cords, candidate_points):
    point_distances = []
    for point in candidate_points:
        #find the closest node coordinate in the path graph to this point
        cc = sorted(all_cords, key=lambda coord: distance(coord, point))[0]
        #note down the point and the distance to its closest coordinate
        point_distances.append((cc, point, distance(cc, point)))
    
    #return the pair that has the smallest distance
    return sorted(point_distances, key=lambda record: record[2])[0]

def get_features_coords(features):
    return [feat['center'] for feat in features]

def location_name_to_coord(g, locations):
    '''Using the geocoding api, map locations to their most likely lat/lon'''
    #only return results within the specific bounding box
    bounding_box = '-95.13478080553124%2C39.119757307389875%2C-70.72830423014456%2C52.52603829818719'
    access_token = 'pk.eyJ1IjoiZmpvaG5zODg4IiwiYSI6ImNsaGh6dnh2ajAzNDkzcXM1OWxwOWF1amIifQ._g2Uwo_6wLTw12W5R-b57w'
    req_url = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?bbox={bounding_box}&access_token={access_token}'
    mapping = {}
    all_coords = coordinates_set(g)
    
    for loc in locations:
        features = requests.get(req_url.format(urllib.parse.quote(loc))).json()
        assert len(features['features']), features
        
        #Use this instead to compare all returned search resutls against graph coordinates
        #use the pair that has the minimum distance from one another
        #candidate_points = get_features_coords(features['features'])
        #cc = closest_coordinate(all_coords, candidate_points)
        #mapping[loc] = {'graphcoord':cc[0], 'geocoded_coord':cc[1]}
        
        geocode = features['features'][0] #get the first and most relevant result
        cc = closest_coordinate(all_coords, [geocode['center']])
        mapping[loc] = {'graphcoord':cc[0], 'geocoded_coord':cc[1], 'geocode_name': geocode['place_name']}
    
    return mapping

def coord_to_node(coordinate):
    for node in g.nodes:
        if g.nodes[node]['coordinates'] == coordinate:
            return node
    raise ValueError("Could not find node matching given coordinate")

Create the graph representation of the geojson data. Create mappings using the Mapbox geocoding api that translate place names to lon/lat.

In [12]:
location_strings = set(df['Where Bound'])
location_strings.update(df['Where From'])
location_strings = sorted(location_strings)

#Exclude the lone multiline path in the geojson data
#this is okay since it its a series of disconnectd strings anyways
features = feature_filter(gj, [837], exclude=True)
g = geojson_to_graph(features)
loc_cord_map = location_name_to_coord(g, location_strings)
loc_node_map = {loc:coord_to_node(loc_cord_map[loc]['graphcoord']) for loc in location_strings}


In [16]:
loc_cord_map

{'11 Mile Creek': {'graphcoord': (-81.67191, 41.4785200000001),
  'geocoded_coord': [-81.28006, 39.53112],
  'geocode_name': '11 Heldman Road, Wingett Run, Ohio 45789, United States'},
 '20 Mile Creek': {'graphcoord': (-83.9469, 43.4246800000001),
  'geocoded_coord': [-83.9985, 43.094929],
  'geocode_name': '20 Six Mile Creek Road, New Lothrop, Michigan 48460, United States'},
 'Allanburg': {'graphcoord': (-79.21033, 43.06744),
  'geocoded_coord': [-79.210718, 43.076399],
  'geocode_name': 'Allanburg Bridge, Port Robinson, Ontario L0S 1K0, Canada'},
 'Amherstburg': {'graphcoord': (-83.138339, 42.1102890000001),
  'geocoded_coord': [-83.108646, 42.101332],
  'geocode_name': 'Amherstburg, Ontario, Canada'},
 'Ashley': {'graphcoord': (-79.54523, 43.58304),
  'geocoded_coord': [-79.71954, 43.636983],
  'geocode_name': "Ashley's No Frills, 7070 McLaughlin Rd, Mississauga, Ontario L5W 1W7, Canada"},
 'Ashtabula': {'graphcoord': (-80.797999, 41.8917470000001),
  'geocoded_coord': [-80.789809,

Some functions that output feature collections (geojson) of the locations that were previously just strings. Basically just used to verify locations that were looked up using the geocoding api make sense.

In [13]:
def mapping_to_geojson(mapping):
    '''Dump location lon/dat data'''
    
    features = {
      "type": "FeatureCollection",
      "features": []
    }
    for loc in mapping:
        loc_gj = {"type": "Feature",
                  "id": loc,
                  "geometry": {
                      "type": "Point", 
                      "coordinates": mapping[loc]['graphcoord']},
                  "properties": {"name": loc}}
        features['features'].append(loc_gj)
    return json.dumps(features)

def mapping_to_geojson_debug(mapping):
    '''Dump queried/looked up lon/lat locations with their most likely graph locations.
    Provided as a LineString so that the discrepancy can be observed'''
    
    linefeatures = {
      "type": "FeatureCollection",
      "features": []
    }
    estfeatures = {
      "type": "FeatureCollection",
      "features": []
    }
    realfeatures = {
      "type": "FeatureCollection",
      "features": []
    }
    features = {}
    
    for loc in mapping:
        loc_gj = {"type": "Feature",
                  "id": loc,
                  "geometry": {
                      "type": "LineString", 
                      "coordinates": [mapping[loc]['graphcoord'], mapping[loc]['geocoded_coord']]},
                  "properties": {"name": loc}}
        est = {"type": "Feature",
               "id": loc,
               "geometry":{"type":"Point", "coordinates":mapping[loc]['graphcoord']},
               "properties":{"name":loc}}
        real = {"type": "Feature",
               "id": loc,
               "geometry":{"type":"Point", "coordinates":mapping[loc]['geocoded_coord']},
               "properties":{"name":mapping[loc]['geocode_name']}}
        linefeatures['features'].append(loc_gj) 
        estfeatures['features'].append(est)
        realfeatures['features'].append(real)
    
    features['lines'] = linefeatures
    features['estimation'] = estfeatures
    features['real'] = realfeatures
    return json.dumps(features)


In [14]:
debug = mapping_to_geojson_debug(loc_cord_map)
with open('debug.json','w') as debug_file:
    debug_file.write(debug)

Code for navigating the created graph representation and deriving the shortest paths for the 'Where From' and 'Where Bound' columns of the CSV

In [15]:
def path_to_coordinates(g, path):
    return [g.nodes[node]['coordinates'] for node in path]

def coordinates_to_geojson(coordinates):
    geo_json = {"type": "Feature",
                "properties":{},
                "geometry":{
                    "type":"LineString",
                    "coordinates":[]
                }}
                
    
    for coordinate in coordinates:
        geo_json['geometry']['coordinates'].append(list(coordinate))
    
    return geo_json

def shortest_path_to_geojson(g, start, end):
    if isinstance(start, int): 
        start = str(start)
    if isinstance(end, int):
        end = str(end)
        
    sp = nx.shortest_path(g,start,end, weight='weight')
    return coordinates_to_geojson(path_to_coordinates(g, sp))

def shortest_paths_to_geojson(g, start_end_pairs):
    '''Given a list of star/end pairs, find the shortest path and return it as a series of linestrings in geojson'''
    geo_json = {"type":"FeatureCollection",
                "features":[]
               }
    geo_json['features'] = [shortest_path_to_geojson(g, start, end) for start, end in start_end_pairs]
    return geo_json

def gen_all_paths(g, df, loc_node_map):
    paths = []
    bad_paths = set()
    for _,locs in df[['Where From','Where Bound']].drop_duplicates().iterrows():
        start_name = locs['Where From']
        dst_name = locs['Where Bound']
        node_start = loc_node_map[start_name]
        node_end = loc_node_map[dst_name]
        path_name = f"{start_name}+{dst_name}"
        try:
            feature = shortest_path_to_geojson(g, node_start, node_end)
            feature['properties']['path'] = path_name
            paths.append(feature)
        except nx.NetworkXNoPath:
            bad_paths.add(path_name)
    
    feature_collection = {"type":"FeatureCollection", "features":paths}
    return json.dumps(feature_collection), bad_paths
good,bad = gen_all_paths(g, df, loc_node_map)
assert not bad, bad
    
# Save cleaned dataframe as JSON    
manifest = df.to_json(orient="records")

with open('manifest.json','w') as manifest_file:
    manifest_file.write(f'{{"manifest":{manifest},\n "routes":{good}}}')
    
#loc_node_map        
# with open('path.json', 'w') as pf:
#     pf.write(shortest_paths_to_geojson(g, [(63100, 300830), (5050, 65450), (490,62870), (62570,300730), (67050, 64470)]))
#shortest_path_to_geojson(g, 63100, 300830)
#g.nodes['64470']['coordinates']
#path_distance(g,sp)

In [ ]:
bad = {'Bear Creek+Clayton', 'Kingston+Bear Creek', 'bear Creek+Kingston', 'Port Hope+Bear Creek', 'Toronto+Bear Creek', 'Port Dalhousie+Bear Creek', 'Hamilton+Bear Creek', 'Oakville+Bear Creek', 'Oswego+Miloin', 'Clayton+Bear Creek', 'Bear Creek+Kingston', 'Port Metcalf+Bear Creek', 'Bear Creek+Ogdensburg', 'Kingston+bear Creek', 'St. Catharines+Bear Creek', 'Kingston+Port Price'}
newbad = set()
for item in bad:
    newbad.update(item.split('+'))
newbad

In [ ]:
g.edges[('226020', '7_1')]

In [ ]:
g.nodes['82_1'],g['82_1'],g.nodes['11300']

In [ ]:
for (u, v, wt) in g.edges.data('weight'):
    print(u,v,wt)

In [ ]:
sum([tup[2] for tup in g.edges.data('weight')])

In [ ]:
sp = nx.shortest_path(g,'63100','62290', weight='weight')

In [34]:
df['Date']

0       1854-04-04
1       1854-04-04
2       1854-04-05
3       1854-04-05
4       1854-04-06
           ...    
1944    1854-11-24
1945    1854-11-24
1946    1854-11-25
1947    1854-11-27
1948    1854-12-01
Name: Date, Length: 1852, dtype: object

In [35]:
manifest = df.to_json(orient="records")

with open('manifest.json','w') as manifest_file:
    manifest_file.write(f'{{"manifest":{manifest},\n "routes":{good}}}')